In [2]:
import pandas as pd
import numpy as np
import sys,os
this_dir = os.path.abspath('.')
parent_dir = os.path.abspath('..')
if this_dir not in sys.path:
    sys.path.append(parent_dir)
    sys.path.append(this_dir)
import db_info#@UnresolvedImport
import typing
import pandas_datareader.data as pdr
import datetime
import tqdm
import plotly_utilities as pu



In [3]:
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'

StrList = typing.List[str]
class SqlDownloader():
    def __init__(self,config_name:str='local'):
        self.pga = db_info.get_db_info(config_name=config_name)
    
    def get_sql(
        self,
        columns=None,
        table=futtab,
        where_clauses=None,
        groupby_columns=None,
        orderby_columns=None,
        limit=None
    ):
        cols = '*' if columns is None else ','.join(columns)
        sql = f"select {cols} from {table}"
        
        if where_clauses is not None:
            wc_string = f'where {where_clauses[0]}'
            if len(where_clauses)>1:
                wc_string =  wc_string + " and " + ' and '.join(where_clauses[1:])
            wc_string = wc_string.replace('%','%%')
            sql = sql + " " + f"{wc_string}"
        if groupby_columns is not None:
            sql += f" group by {','.join(groupby_columns)}"
        if orderby_columns is not None:
            sql += f" order by {','.join(orderby_columns)}"
        if limit is not None:
            sql += f" limit {int(str(limit))}"
        df = self.pga.get_sql(sql)
        return df
        
    def get_options(self,symbol:str,yyyymmdd_beg:int=None,yyyymmdd_end:int=None):
        sql = f"select * from {opttab} where symbol='{symbol}'"
        if yyyymmdd_beg is not None:
            sql += f' and settle_date>={yyyymmdd_beg}'
        if yyyymmdd_end is not None:
            sql += f' and settle_date<={yyyymmdd_end}'
        df =  self.pga.get_sql(sql)
        if df is not None and len(df)>2:
            df = df.sort_values(['pc','settle_date','strike'])
        return df

    def get_futures(self,symbol_list:StrList,yyyymmdd_beg:int=None,yyyymmdd_end:int=None):
        if type(symbol_list) is list or type(symbol_list) is np.ndarray:
            sym_string = ",".join([f"'{s}'" for s in symbol_list]) 
        else:
            sym_string = f"'{symbol_list}'"
        sql = f"select * from {futtab} where symbol in ({sym_string})"
        if yyyymmdd_beg is not None:
            sql += f' and settle_date>={yyyymmdd_beg}'
        if yyyymmdd_end is not None:
            sql += f' and settle_date<={yyyymmdd_end}'
        df =  self.pga.get_sql(sql)
        if df is not None and len(df)>2:
            df = df.sort_values(['symbol','settle_date'])
        return df


In [4]:
sqdl = SqlDownloader()

  sec_db


In [5]:
cols = ['distinct symbol']
where_colummns = ["symbol like 'CL%'"]
df_symbols = sqdl.get_sql(columns=cols,where_clauses=where_colummns)

In [6]:
syms = df_symbols.symbol.values


In [7]:
syms2  = sorted([s[0:2]+s[-2:]+s[2] for s in syms])
syms2 = [s for s in syms2 if s<='CL21F']
syms3 = [s[0:2]+s[-1]+s[2:4] for s in syms2]
syms3

['CLV10',
 'CLX10',
 'CLZ10',
 'CLF11',
 'CLG11',
 'CLH11',
 'CLJ11',
 'CLK11',
 'CLM11',
 'CLN11',
 'CLQ11',
 'CLU11',
 'CLV11',
 'CLX11',
 'CLZ11',
 'CLF12',
 'CLG12',
 'CLH12',
 'CLJ12',
 'CLK12',
 'CLM12',
 'CLN12',
 'CLQ12',
 'CLU12',
 'CLV12',
 'CLX12',
 'CLZ12',
 'CLF13',
 'CLG13',
 'CLH13',
 'CLJ13',
 'CLK13',
 'CLM13',
 'CLN13',
 'CLQ13',
 'CLU13',
 'CLV13',
 'CLX13',
 'CLZ13',
 'CLF14',
 'CLG14',
 'CLH14',
 'CLJ14',
 'CLK14',
 'CLM14',
 'CLN14',
 'CLQ14',
 'CLU14',
 'CLV14',
 'CLX14',
 'CLZ14',
 'CLF15',
 'CLG15',
 'CLH15',
 'CLJ15',
 'CLK15',
 'CLM15',
 'CLN15',
 'CLQ15',
 'CLU15',
 'CLV15',
 'CLX15',
 'CLZ15',
 'CLF16',
 'CLG16',
 'CLH16',
 'CLJ16',
 'CLK16',
 'CLM16',
 'CLN16',
 'CLQ16',
 'CLU16',
 'CLV16',
 'CLX16',
 'CLZ16',
 'CLF17',
 'CLG17',
 'CLH17',
 'CLJ17',
 'CLK17',
 'CLM17',
 'CLN17',
 'CLQ17',
 'CLU17',
 'CLV17',
 'CLX17',
 'CLZ17',
 'CLF18',
 'CLG18',
 'CLH18',
 'CLJ18',
 'CLK18',
 'CLM18',
 'CLN18',
 'CLQ18',
 'CLU18',
 'CLV18',
 'CLX18',
 'CLZ18',
 'CLF19',


In [8]:
orderby_columns = [
    "settle_date desc"
]
dict_df = {}
for symbol in tqdm.tqdm(syms3):
    where_clauses = [f"symbol='{symbol}'"]
    df = sqdl.get_sql(where_clauses=where_clauses,orderby_columns=orderby_columns,limit=30)
    dict_df[symbol] = df.copy()

100%|██████████| 124/124 [00:00<00:00, 347.08it/s]


In [10]:
first_sym = [s for s in dict_df.keys()][0]
first_yyyymmdd = dict_df[first_sym].iloc[0].settle_date
last_sym = [s for s in dict_df.keys()][-1]
last_yyyymmdd = dict_df[last_sym].iloc[-1].settle_date

first_yyyymmdd,last_yyyymmdd

(20100921, 20201109)

In [11]:
def get_pdr(symbol,beg_dt=None,end_dt=None,num_days=None,data_source='yahoo'):
    edt = datetime.datetime.now() if end_dt is None else end_dt
    if beg_dt is None:
        numd = 365 if num_days is None else num_days
        bdt = edt - datetime.timedelta(numd)
    else:
        bdt = beg_dt
    df = pdr.DataReader(symbol,start=bdt,end=edt,data_source=data_source)
    df['Date'] = df.index
    df.index = list(range(len(df)))
    df['yyyymmdd'] = df.Date.apply(lambda v:int(str(v)[0:4])*100*100 + int(str(v)[5:7])*100 + int(str(v)[8:10]))
    return df

def get_pdr_yyyymmdd(symbol,beg_yyyymmdd,end_yyyymmdd):
    beg_yyyy = int(str(beg_yyyymmdd)[0:4])
    beg_mm = int(str(beg_yyyymmdd)[4:6])
    beg_dd = int(str(beg_yyyymmdd)[6:8])
    beg_dt = datetime.datetime(beg_yyyy,beg_mm,beg_dd)
    end_yyyy = int(str(end_yyyymmdd)[0:4])
    end_mm = int(str(end_yyyymmdd)[4:6])
    end_dd = int(str(end_yyyymmdd)[6:8])
    end_dt = datetime.datetime(end_yyyy,end_mm,end_dd)
    return get_pdr(symbol,beg_dt = beg_dt,end_dt = end_dt)
    

In [12]:
df_xle = get_pdr_yyyymmdd('XLE',first_yyyymmdd,last_yyyymmdd)

In [13]:
df_xle = df_xle.rename(columns={'yyyymmdd':'settle_date'})

In [14]:
df_xle2 = df_xle[['Close','settle_date']]
df_xle2 = df_xle2.rename(columns={'Close':'XLE'})

In [90]:
df_z99 = sqdl.get_futures('CLZ99',yyyymmdd_beg=df_all.settle_date.min(),yyyymmdd_end = df_all.settle_date.max())
df_z99 = df_z99[['settle_date','close']].rename(columns={'close':'z99'})


In [15]:
dict_df['CLF19']

,symbol,settle_date,contract_num,open,high,low,close,adj_close,volume,open_interest
0,CLF19,20181219,1,45.93,48.00,45.93,47.20,47.20,120101,29550
1,CLF19,20181218,1,49.16,49.59,45.79,46.24,46.24,211257,66901
2,CLF19,20181217,1,51.25,51.87,49.01,49.88,49.88,526839,90329
3,CLF19,20181214,1,52.83,52.95,50.84,51.20,51.20,733627,146352
4,CLF19,20181213,1,51.20,53.27,50.35,52.58,52.58,730810,186783
5,CLF19,20181212,1,51.95,52.88,50.94,51.15,51.15,665262,235703
6,CLF19,20181211,1,50.89,52.43,50.70,51.65,51.65,733225,278005
7,CLF19,20181210,1,52.03,52.81,50.53,51.00,51.00,987717,324223
8,CLF19,20181207,1,51.76,54.22,50.60,52.61,52.61,987266,363183
9,CLF19,20181206,1,52.93,53.30,50.08,51.49,51.49,624914,380601


### create a binary strategy whereby
you predict "up" or "down" in CL each day from the up/down value of XLE

In [106]:
dict_df2 = {}
for symbol in tqdm.tqdm(syms3):
    # get CL and rename 'close' to symbol
    df = dict_df[symbol][['settle_date','close']]
    df = df.rename(columns={'close':symbol})
    # merge CL and XLE
    dfb = df.merge(df_xle2,on='settle_date',how='inner')
    # get perc change per day of XLE
    dfb[symbol] = dfb[symbol].pct_change()
    dfb.XLE = dfb.XLE.pct_change()
    # create an indicator that determines if we make a coin flip bet that 
    #   CL goes up or down the next day
    dfb['updown_prediction'] = dfb.XLE.shift(1)/dfb.XLE.shift(1).abs()
    dfb['updown_result'] = dfb[symbol]/dfb[symbol].abs() * dfb.updown_prediction
    dfb['reverse_prediction'] = dfb[symbol].shift(1)/dfb[symbol].shift(1).abs()
    dfb['reverse_result'] = dfb.XLE/dfb.XLE.abs() * dfb.updown_prediction
    dict_df2[symbol] = dfb[~dfb.updown_result.isna()].copy()

100%|██████████| 124/124 [00:01<00:00, 87.83it/s]


In [107]:
total=0
tries=0
df_all = None
for symbol in tqdm.tqdm(syms3):
    df = dict_df2[symbol].copy()
    total += df.updown_result.sum()
    tries += len(df)
    df = df.rename(columns={symbol:'close'})
    if df_all is None:
        df_all = df.copy()
    else:
        df_all = df_all.append(df,ignore_index=True)

# get counts of the number of updown result rows there are for each settle_date, b/c
#   some of the settle_dates have multiple futures contracts in df_all
df_counts = df_all[['settle_date','updown_result']].groupby('settle_date').agg(['count',sum])
df_counts.columns = df_counts.columns.get_level_values(1)
df_all = df_all.merge(df_counts,on='settle_date')
# this next line adjusts updown_result to account for settle_date's when there is more than one contract
#   that has an updown_result.  We divide the updown_result by the number of contracts for that settle_date
df_all.updown_result = df_all.updown_result/df_all['count']
df_all.reverse_result = df_all.reverse_result/df_all['count']
df_all['cum_updown_result'] = df_all.updown_result.cumsum()
df_all['cum_reverse_result'] = df_all.reverse_result.cumsum()
df_all = df_all.merge(df_z99,on='settle_date',how='inner')

df_xle_close_only = df_xle[['settle_date','Close']]
df_xle_close_only = df_xle_close_only.rename(columns={'Close':'xle_close'})
df_all = df_all.merge(df_xle_close_only,on='settle_date',how='inner')

all_sum = df_all.updown_result.sum()
all_len = len(df_all.settle_date.unique())
all_perc = all_sum / all_len
all_std_daily = df_all.updown_result.std()
all_std_yearly = all_std_daily * 256**.5
all_perc_yearly = all_perc * 256
yearly_sharpe = all_perc_yearly/all_std_yearly
{
    'total_profit':all_sum,
    'num_days':all_len,
    'daily_avg_return':all_perc,
    'yearly_profit':all_perc_yearly,
    'all_std_yearly':all_std_yearly,
    'yearly_sharpe':yearly_sharpe    
}


100%|██████████| 124/124 [00:00<00:00, 561.06it/s]


{'total_profit': -150.0,
 'num_days': 2529,
 'daily_avg_return': -0.05931198102016608,
 'yearly_profit': -15.183867141162516,
 'all_std_yearly': 12.668086829119849,
 'yearly_sharpe': -1.1985919694092797}

In [108]:
df_all.updown_result.std()


0.7917554268199906

In [116]:
import importlib
importlib.reload(pu)
results = []
reverse_results = []
for symbol in tqdm.tqdm(syms3):
    results.append(dict_df2[symbol].updown_result.sum())    
    reverse_results.append(dict_df2[symbol].reverse_result.sum())    
df_results = pd.DataFrame({'symbol':syms3,'result':results,'reverse_result':reverse_results}) 
df_results['cumul_result'] = df_results.result.cumsum() 
df_results['cumul_reverse'] = df_results.reverse_result.cumsum() 
pu.iplt(
    pu.plotly_plot(
        df_results,x_column='symbol',
        yaxis2_cols=['cumul_result','cumul_reverse'],
        go_functions=[pu.go.Bar,pu.go.Bar,pu.go.Scatter,pu.go.Scatter],
        opacity_list=[.75,.75,1,1]
    )
)

100%|██████████| 124/124 [00:00<00:00, 2807.40it/s]


In [113]:
df_results

,symbol,result,reverse_result,cumul_result,cumul_reverse
0,CLV10,0.0,0.0,0.0,0.0
1,CLX10,-4.0,-6.0,-4.0,-6.0
2,CLZ10,-1.0,3.0,-5.0,-3.0
3,CLF11,4.0,-2.0,-1.0,-5.0
4,CLG11,4.0,-2.0,3.0,-7.0
...,...,...,...,...,...
119,CLU20,-3.0,-5.0,-211.0,-98.0
120,CLV20,2.0,2.0,-209.0,-96.0
121,CLX20,-6.0,-2.0,-215.0,-98.0
122,CLZ20,-2.0,-3.0,-217.0,-101.0


In [110]:
pu.iplt(
    pu.plotly_plot(
        df_all[['settle_date','cum_updown_result','z99','xle_close']],
        x_column='settle_date',
        yaxis2_cols=['z99','xle_close']
    )
)

In [42]:
dict_pu = {}
for symbol in tqdm.tqdm(syms3):
    df = dict_df2[symbol]
    try:
        f = pu.plotly_plot(
            df[['settle_date','updown_result']],
            x_column='settle_date',
            bar_plot=True
        )
        dict_pu[symbol] = f
    except:
        pass

100%|██████████| 124/124 [00:23<00:00,  5.31it/s]


In [44]:
dict_pu['CLF12']